# STATS401 Project 2


## Data Acquisition
---

### Query the transaction data of CryptoPunk from [Dune Analytics](https://dune.com/browse/queries/authored)


In [ ]:
!pip install duneanalytics


^C
ERROR: Operation cancelled by user


In [ ]:
from duneanalytics import DuneAnalytics
import time
import json
import pandas as pd
import numpy as np


In [ ]:
def query_data_from_dune(data_mode):
    """
    Query data from Dune Analytics
    """
    # initialize client
    dune = DuneAnalytics('brucezzzzzzzyf@gmail.com', 'ZHANG0509yf')

    # try to login
    dune.login()

    # fetch token
    dune.fetch_auth_token()

    # set query ids
    if data_mode == 'punkBought':
        query_dict = {'2017': 673785,
                      '2018': 673784,
                      '2019': 673783,
                      '2020': 673781,
                      '2021': 673779,
                      '2022': 673771}
    elif data_mode == 'punkTransfer':
        query_dict = {'2017-2019': 674456,
                      '2020': 674472,
                      '2021': 674496,
                      '2022': 674500}

    # query data
    dune_data = {}
    for year in query_dict:
        query_id = query_dict[year]
        result_id = dune.query_result_id(query_id=query_id)
        dune_data[year] = dune.query_result(result_id)
        time.sleep(5)

    return dune_data


def save_data(dune_dict, file_predix, DUNE_DATA_PATH='./dune_data'):
    """
    Write data to json file
    """
    from datetime import date
    today = str(date.today())
    out_file = '{}/{}_{}.json'.format(DUNE_DATA_PATH, file_predix, today)
    with open(out_file, 'w') as f:
        json.dump(dune_dict, f)
    print('Data saved to {}'.format(out_file))


In [ ]:
dune_bt_dict = query_data_from_dune('punkBought')
save_data(dune_bt_dict, 'dune_bt_dict')


{'data': {'get_result': {'job_id': None, 'result_id': 'd9007449-6c39-48fa-8731-f58874ee704c', '__typename': 'get_result_response'}}}
{'data': {'query_results': [{'id': 'd9007449-6c39-48fa-8731-f58874ee704c', 'job_id': '8b473fc4-3325-446f-9c7f-1a25b55ce77d', 'error': None, 'runtime': 0, 'generated_at': '2022-05-06T20:36:57.499385+00:00', 'columns': ['_date', '_time', 'seller', 'buyer', 'punk_id', 'eth_price'], '__typename': 'query_results'}], 'get_result_by_result_id': [{'data': {'_date': '2017-12-30', '_time': '04:12:23', 'buyer': '\\x0000000000000000000000000000000000000000', 'eth_price': 0.0, 'punk_id': 6558, 'seller': '\\x00d7c902fbbcd3c9db2da80a439c94486c50eb81'}, '__typename': 'get_result_template'}, {'data': {'_date': '2017-12-29', '_time': '22:12:07', 'buyer': '\\x6a8920d18b477c3e7446cf7f82dc0a9e38bf5fcf', 'eth_price': 0.1, 'punk_id': 8215, 'seller': '\\x00d7c902fbbcd3c9db2da80a439c94486c50eb81'}, '__typename': 'get_result_template'}, {'data': {'_date': '2017-12-29', '_time': '1

KeyboardInterrupt: 

In [ ]:
dune_tf_dict = query_data_from_dune('punkTransfer')
save_data(dune_tf_dict, 'dune_tf_dict')


## Data Cleaning
---

In [ ]:
import time
import json
import pandas as pd
import numpy as np

In [ ]:
dune_bt_dict = json.load(open('./dune_data/dune_bt_dict_2022-05-06.json'))
dune_tf_dict = json.load(open('./dune_data/dune_tf_dict_2022-05-06.json'))

In [ ]:
def index_trader(buyer, seller):
    trader = list(set(list(buyer.unique()) + list(seller.unique())))
    # print('Total {} traders'.format(len(trader)))

    return {trader[i]: i for i in range(len(trader))}


### 1. Transaction Database

- **tx_id**
- date_time
- from: trader_id
- yo: trader_id
- eth_price
- punk_id: punk_id


In [ ]:
def create_tx_db(dune_bt_dict, dune_tf_dict):
    temp_idx = 0
    # punkBought data frame
    bt_data = dict()
    for year in dune_bt_dict:
        data_list = dune_bt_dict[year]['data']['get_result_by_result_id']
        bt_data.update({temp_idx+i: data_list[i]['data']
                       for i in range(len(data_list))})
        temp_idx += len(data_list)

    # punkTransfer data frame
    tf_data = dict()
    for year in dune_tf_dict:
        data_list = dune_tf_dict[year]['data']['get_result_by_result_id']
        tf_data.update({temp_idx+i: data_list[i]['data']
                       for i in range(len(data_list))})
        temp_idx += len(data_list)

    # concatenate data frames
    tx_db = pd.concat([pd.DataFrame.from_dict(bt_data, orient='index'),
                       pd.DataFrame.from_dict(tf_data, orient='index')])

    # set date_time
    tx_db['date_time'] = pd.to_datetime(tx_db['_date'] + ' ' + tx_db['_time'])

    # remove txs whose buyer is \x0000000000000000000000000000000000000000
    tx_db = tx_db[tx_db['buyer'] !=
                  '\\x0000000000000000000000000000000000000000']

    # reindex seller and buyer using trader_id
    trader_index_dict = index_trader(tx_db['buyer'], tx_db['seller'])
    tx_db['from'] = tx_db['seller'].apply(lambda x: trader_index_dict[x])
    tx_db['to'] = tx_db['buyer'].apply(lambda x: trader_index_dict[x])

    # sort by date_time
    tx_db.sort_values(by='date_time', inplace=True)

    # index transaction
    tx_db['tx_id'] = range(len(tx_db))
    tx_db = tx_db.loc[:, ['tx_id', 'date_time',
                          'from', 'to', 'eth_price', 'punk_id']]
    tx_db.set_index('tx_id', inplace=True)

    print('Total {} transactions'.format(len(tx_db)))
    print('Total {} unique traders'.format(len(trader_index_dict)))
    print('Total {} unique punk_id'.format(len(tx_db['punk_id'].unique())))

    return tx_db, trader_index_dict


tx_db, trader_index_dict = create_tx_db(dune_bt_dict, dune_tf_dict)
tx_db


Total 40540 transactions
Total 8907 unique traders
Total 9007 unique punk_id


,date_time,from,to,eth_price,punk_id
tx_id,,,,,
0,2017-06-23 21:06:32,8703,7678,0.01,3134
1,2017-06-23 21:06:45,8703,2734,0.04,5719
2,2017-06-23 21:06:53,7678,4191,0.10,5056
3,2017-06-23 22:06:04,8703,4439,NaN,8138
4,2017-06-23 22:06:08,8703,2734,0.06,5624
...,...,...,...,...,...
40535,2022-05-05 21:05:57,3576,6799,70.00,3314
40536,2022-05-06 02:05:19,5134,2536,NaN,3904
40537,2022-05-06 03:05:13,1310,1407,NaN,4332


### 2. CryptoPunk

- **punk_id**
- img_url
- type (Alien, Ape, Zombie, Female, Male)
- skin_tone
- attr_count
- attributes
- current_owner: trader_id
- avg_price


In [ ]:

def create_punk_db(punk_info_csv_list, CSV_PATH='./cp', PUNK_IMG_PATH='./punk_imgs'):
    punk_db = pd.read_csv('{}/{}'.format(CSV_PATH, punk_info_csv_list[0]))
    for i in range(1, len(punk_info_csv_list)):
        punk_df_temp = pd.read_csv(
            '{}/{}'.format(CSV_PATH, punk_info_csv_list[i]))
        punk_db = pd.concat([punk_db, punk_df_temp], axis=0)

    # rename columns
    punk_db.columns = ['punk_id', 'type', 'gender',
                       'skin_tone', 'attr_count', 'attributes']

    # strip type, gender, skin_tone
    punk_db['type'] = punk_db['type'].apply(lambda x: x.strip())
    punk_db['gender'] = punk_db['gender'].apply(lambda x: x.strip())
    punk_db['skin_tone'] = punk_db['skin_tone'].apply(lambda x: x.strip())
    
    # set cryptopunk image dir
    punk_db['img_dir'] = punk_db['punk_id'].apply(
        lambda x: '{}/{}.png'.format(PUNK_IMG_PATH, x))
    
    # make attributes as list
    punk_db['attributes'] = punk_db['attributes'].apply(lambda x: [i.strip() for i in x.split('/')])

    # set punk_id as index
    punk_db.set_index('punk_id', inplace=True)

    return punk_db


punk_info_csv_list = ['0-999.csv', '1000-1999.csv', '2000-2999.csv', '3000-3999.csv',
                      '4000-4999.csv', '5000-5999.csv', '6000-6999.csv',
                      '7000-7999.csv', '8000-8999.csv', '9000-9999.csv']

punk_db = create_punk_db(punk_info_csv_list)
punk_db


,type,gender,skin_tone,attr_count,attributes,img_dir
punk_id,,,,,,
0,Human,Female,Medium,3,"[Green Eye Shadow, Earring, Blonde Bob]",./punk_imgs/0.png
1,Human,Male,Dark,2,"[Smile, Mohawk]",./punk_imgs/1.png
2,Human,Female,Light,1,[Wild Hair],./punk_imgs/2.png
3,Human,Male,Dark,3,"[Wild Hair, Nerd Glasses, Pipe]",./punk_imgs/3.png
4,Human,Male,Medium,4,"[Big Shades, Wild Hair, Earring, Goat]",./punk_imgs/4.png
...,...,...,...,...,...,...
9995,Human,Female,Albino,2,"[Purple Eye Shadow, Straight Hair Dark]",./punk_imgs/9995.png
9996,Human,Male,Light,4,"[Cigarette, Earring, Crazy Hair, Smile]",./punk_imgs/9996.png
9997,Zombie,Male,,2,"[Front Beard, Cap Forward]",./punk_imgs/9997.png


### 3. Trader

- **trader_id**
- address
- tx_involved_count
- tx_involved: list of tx_id
- frequent_skin_tone
- frequent_gender


In [ ]:

def find_most_frequent_skin_tone(tx_involved_list, tx_db):
    skin_tones = list()
    for tx_id in tx_involved_list:
        punk_id = tx_db.loc[tx_id, 'punk_id']
        skin_tones.append(punk_db.loc[punk_id, 'skin_tone'])
    return max(set(skin_tones), key=skin_tones.count)


def find_most_frequent_gender(tx_involved_list, tx_db):
    genders = list()
    for tx_id in tx_involved_list:
        punk_id = tx_db.loc[tx_id, 'punk_id']
        genders.append(punk_db.loc[punk_id, 'gender'])
    return max(set(genders), key=genders.count)


def create_trader_db(tx_db, trader_index_dict):
    trader_db = pd.DataFrame.from_dict(
        trader_index_dict, orient='index', columns=['trader_id'])

    # set address for each trader
    trader_db.reset_index(inplace=True)
    trader_db.rename(columns={'index': 'address'}, inplace=True)

    # set trader_id as index
    trader_db.sort_values(by='trader_id', inplace=True)

    # get the involved tx
    trader_db['tx_involved_count'] = trader_db['trader_id'].apply(
        lambda x: len(tx_db[tx_db['from'] == x]) + len(tx_db[tx_db['to'] == x]))
    trader_db['tx_involved'] = trader_db['trader_id'].apply(lambda x: sorted(list(tx_db[tx_db['from'] == x].index) +
                                                            list(tx_db[tx_db['to'] == x].index)))

    # get the skin tone that each trader most frequently trades with
    trader_db['frequent_skin_tone'] = trader_db['tx_involved'].apply(
        lambda x: find_most_frequent_skin_tone(x, tx_db))

    # get the gender that each trader most frequently trades with
    trader_db['frequent_gender'] = trader_db['tx_involved'].apply(
        lambda x: find_most_frequent_gender(x, tx_db))

    # set trader_id as index
    trader_db.set_index('trader_id', inplace=True)

    return trader_db


trader_db = create_trader_db(tx_db, trader_index_dict)
trader_db


,address,tx_involved_count,tx_involved,frequent_skin_tone,frequent_gender
trader_id,,,,,
0,\xc4eab1eaacbf628f0f9aee4b7375bde18dd173c4,9,"[1411, 1480, 1485, 1486, 2313, 4905, 4988, 517...",Dark,Male
1,\x5e49d06e618680e8064a88e63c12c873f52e1f28,3,"[14846, 14852, 15852]",Light,Male
2,\x72915ad3110eb31768a562f540ac1ebcd51d3dc8,2,"[29106, 30262]",Light,Male
3,\x957f182a6e475c8d89395d9964baf78691d93c2a,2,"[2265, 31332]",Medium,Female
4,\x6403d0e123c9e0f73265ab3f3dfd4c8ba3599f85,2,"[39951, 39952]",Albino,Male
...,...,...,...,...,...
8902,\xf0692d3344987619906f9299c4e0f8f43312db16,2,"[30469, 37763]",Dark,Male
8903,\xb0f96a681e2e1bcbf40f99f0e48b8f3108089cb2,3,"[19461, 20458, 29056]",Medium,Female
8904,\x1224917a739dff52c50b232042cc1b196d2daced,2,"[6428, 6992]",Medium,Female


## Data Engineering for Visualization
---

In [ ]:
VIS_DATA_PATH = './vis_data'

### 1. Sankey Diagram

<img src="imgs/vis1_example.png" alt="drawing" width="500">

Data format

```
{
    "nodes":[
        {"node":0,"name":"Human"},
        {"node":1,"name":"Zombie"},
        {"node":2,"name":"Ape"},
        {"node":3,"name":"Alien"},

        {"node":4,"name":"Female"},
        {"node":5,"name":"Male"},

        {"node":6,"name":"Medium"},
        {"node":7,"name":"Dark"},
        {"node":8,"name":"Light"},
        {"node":9,"name":"Albino"},
        {"node":10,"name":"non-human"},

        {"node":11,"name":"Beanie"},
        ...
        {"node":97,"name":"Stringy Hair"}
    ],

    "links":[
        {"source":<node_id>, "target":<node_id>, "value":<count>},
        ...
    ]
}
```

In [ ]:
def between_skin_tone_and_attributes(item, source_skin_tone, target_attribute):
    return (item['skin_tone'] == source_skin_tone) and (target_attribute in item['attributes'])


def get_dataset_vis1(tx_db, punk_db, trader_db):
    dataset_vis1 = dict()

    # nodes
    node_names_nested_list = list()
    # get the list of all types
    # ['Human', 'Zombie', 'Ape', 'Alien']
    type_list = list(punk_db['type'].unique())
    node_names_nested_list.append(type_list)

    # get the list of all genders
    gender_list = list(punk_db['gender'].unique()
                       )          # ['Female', 'Male']
    node_names_nested_list.append(gender_list)

    # get the list of all skin_stones
    # ['Medium', 'Dark', 'Light', 'Albino', '']
    skin_tone_list = list(punk_db['skin_tone'].unique())
    skin_tone_list.remove('')
    # skin_tone_list = [
    #     x if x != '' else 'Unknown_skin_tone' for x in skin_tone_list]
    node_names_nested_list.append(skin_tone_list)

    # get the list of all attributes
    attribute_list = list(punk_db.attributes)
    attribute_list = list(
        set([item for elem in attribute_list for item in elem]))
    attribute_list = sorted(attribute_list)
    attribute_list.remove('')
    # attribute_list = [
    #     x if x != '' else 'No attributes' for x in attribute_list]
    node_names_nested_list.append(attribute_list)

    # set node_id and node_name
    node_list = list()

    node_names = type_list + gender_list + attribute_list
    for name in node_names:
        node_list.append({'node': len(node_list), 'name': name})

    dataset_vis1['nodes'] = node_list
    print('> Saved: nodes')

    # links
    link_list = list()
    levels_list = ['type', 'gender', 'skin_tone']
    # links between each pair of neighbor levels
    for i in range(len(levels_list)-1):
        source_level = levels_list[i]
        source_name_list = node_names_nested_list[i]

        target_level = levels_list[i+1]
        target_name_list = node_names_nested_list[i+1]

        for source in source_name_list:
            for target in target_name_list:
                value = punk_db[(punk_db[source_level] == source) & (
                    punk_db[target_level] == target)].shape[0]
                link_list.append(
                    {'source': source, 'target': target, 'value': value})
    print('>> Saved: links between type, gender, and skin_tone')

    # links between skin_tone and attributes
    for source in skin_tone_list:
        for target in attribute_list:
            value = punk_db[punk_db.apply(lambda x: between_skin_tone_and_attributes(
                x, source, target), axis=1)].shape[0]
            link_list.append(
                {'source': source, 'target': target, 'value': value})
    print('>> Saved: links between skin_tone and attributes')

    dataset_vis1['links'] = link_list
    print('> Saved: links')

    return dataset_vis1


vis1_data = get_dataset_vis1(tx_db, punk_db, trader_db)
with open('{}/vis1_data.json'.format(VIS_DATA_PATH), 'w') as f:
    json.dump(vis1_data, f)
vis1_data


> Saved: nodes
>> Saved: links between type, gender, and skin_tone
>> Saved: links between skin_tone and attributes
> Saved: links


{'nodes': [{'node': 0, 'name': 'Human'},
  {'node': 1, 'name': 'Zombie'},
  {'node': 2, 'name': 'Ape'},
  {'node': 3, 'name': 'Alien'},
  {'node': 4, 'name': 'Female'},
  {'node': 5, 'name': 'Male'},
  {'node': 6, 'name': '3D Glasses'},
  {'node': 7, 'name': 'Bandana'},
  {'node': 8, 'name': 'Beanie'},
  {'node': 9, 'name': 'Big Beard'},
  {'node': 10, 'name': 'Big Shades'},
  {'node': 11, 'name': 'Black Lipstick'},
  {'node': 12, 'name': 'Blonde Bob'},
  {'node': 13, 'name': 'Blonde Short'},
  {'node': 14, 'name': 'Blue Eye Shadow'},
  {'node': 15, 'name': 'Buck Teeth'},
  {'node': 16, 'name': 'Cap'},
  {'node': 17, 'name': 'Cap Forward'},
  {'node': 18, 'name': 'Chinstrap'},
  {'node': 19, 'name': 'Choker'},
  {'node': 20, 'name': 'Cigarette'},
  {'node': 21, 'name': 'Classic Shades'},
  {'node': 22, 'name': 'Clown Eyes Blue'},
  {'node': 23, 'name': 'Clown Eyes Green'},
  {'node': 24, 'name': 'Clown Hair Green'},
  {'node': 25, 'name': 'Clown Nose'},
  {'node': 26, 'name': 'Cowboy H

### 2. Time River

In [ ]:
data2_punk = punk_db[['type','gender','skin_tone']]
data2_punk.reset_index()
data2_tx = tx_db[['date_time','eth_price','punk_id']]

# merge the tx data with punk data
data2 = pd.merge(data2_punk,data2_tx,on="punk_id",how="outer")
data2_clean = data2[data2.eth_price > 0]
data2_clean['year_month'] = data2_clean['date_time'].dt.to_period('M')

# group the data by gender, skin_tone, and year_month, and compute the mean price
male_light = data2_clean[(data2_clean.gender == 'Male') & ((data2_clean.skin_tone == "Light")) ].groupby("year_month").agg({"eth_price": ["mean"]}).reset_index("year_month")
male_light


/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,year_month,eth_price
,,mean
0,2017-06,0.181507
1,2017-07,0.325373
2,2017-08,0.323492
3,2017-09,0.290000
4,2017-10,0.198571
5,2017-11,0.230000
6,2017-12,0.122762
7,2018-01,0.123269
8,2018-02,0.120000


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=46584e2a-add3-4c70-b004-7268e7caedfc' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>